In [3]:
%load_ext autoreload
%autoreload 2

from data_gen import ExpressionDataset, bert_collate_strs, EXPR_TOKENIZER
import torch
from torch.utils.data import DataLoader, random_split
import pytorch_lightning as pl
from model import BERTModule
from pytorch_lightning.callbacks.progress import TQDMProgressBar

fds = ExpressionDataset.from_csv("data/equation_dataset.csv")
l = len(fds)
a = l *4 //5
b = l//5
c = l - a - b
train_ds, val_ds, test_ds = random_split(fds, [a, b, c], generator=torch.Generator().manual_seed(1))
train_dl = DataLoader(train_ds, shuffle=True, batch_size=256, collate_fn=bert_collate_strs)
val_dl = DataLoader(val_ds, shuffle=False, batch_size=512, collate_fn=bert_collate_strs)



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
val_b = [val_ds[0], val_ds[1], val_ds[2]]
a, b, c, d = next(iter(val_dl))
print(a.shape)

torch.Size([512, 22])


In [15]:
masked_src_id, masked_token_mask, src_id, src_pad_mask = bert_collate_strs(val_b)

print(masked_src_id)
print(masked_token_mask, masked_src_id[masked_token_mask])
print(src_id)

tensor([[ 2, 14,  4, 19, 13, 16, 17,  4,  6],
        [ 2, 11, 15, 11,  6,  0,  0,  0,  0],
        [ 2, 12,  8,  4, 12, 17, 17, 13,  6]])
tensor([[False, False,  True, False, False, False, False,  True, False],
        [False, False,  True, False, False, False, False, False, False],
        [False, False, False,  True, False, False,  True, False, False]]) tensor([ 4,  4, 15,  4, 17])
tensor([[ 2, 14, 15, 19, 13, 16, 17, 16,  6],
        [ 2, 11, 19, 11,  6,  0,  0,  0,  0],
        [ 2, 12,  8, 19, 12, 17, 11, 13,  6]])


In [12]:
print(masked_src_id)
print(src_id)

tensor([[ 2, 14, 15, 21, 13, 16, 17, 16,  6],
        [ 2, 11, 21, 11,  6,  0,  0,  0,  0],
        [ 4, 12,  8, 21, 12, 17, 11, 13,  6]])
tensor([[ 2, 14, 15, 21, 13, 16, 17, 16,  6],
        [ 2, 11, 21, 11,  6,  0,  0,  0,  0],
        [ 2, 12,  8, 21, 12, 17, 11, 13,  6]])


In [6]:
import vocab
print(vocab.MASK_IDX, vocab.CLS_IDX)
print(vocab.EXPR_TOKENIZER.get_vocab())


4 2
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]


In [22]:
print(EXPR_TOKENIZER.encode(val_ds[0]))

Tokenizer(vocabulary_size=19, model=BertWordPiece, unk_token=[UNK], sep_token=[SEP], cls_token=[CLS], pad_token=[PAD], mask_token=[MASK], clean_text=True, handle_chinese_chars=True, strip_accents=None, lowercase=True, wordpieces_prefix=##)


Exception: WordPiece error: Missing [UNK] token from the vocabulary

In [21]:
print(EXPR_TOKENIZER.encode_batch([val_ds[0], val_ds[1], val_ds[2]]))

Exception: WordPiece error: Missing [UNK] token from the vocabulary